In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


![](https://miro.medium.com/max/1400/1*ee5sr4o6lAJ6qcdKXhGoFg.png)

# Introduction

**Purpose of this notebook is to give insights into the dataset, build a Nueral network to identify digits from images in the MNIST dataset**

**Tasks:**

1. To understand datasets( train and test), variables, clean(if needed) the data and visualize important use-cases.

2. Consider 'label' as the independent variable from the train_set.

3. Train a Neural network for a given train-test-split peices, examine epochs, accuracy, re-tune NN's parameters(if needed).

4. Manully predict a few images and record history-plots.

5. Upload the submission.

In [2]:
# Imports

import numpy as ny
import pandas as ps
import plotly.express as plex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import tensorflow as tflow
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

2021-09-28 17:00:09.694971: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-28 17:00:09.695100: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
train_set = ps.read_csv("../input/digit-recognizer/train.csv")
test_set = ps.read_csv("../input/digit-recognizer/test.csv")

# Verifying shapes and values

print(" \n Length of train_set is :", len(train_set)," and shape is: ",train_set.shape)
print(" \n Length of test_set is :", len(test_set)," and shape is: ",test_set.shape)

 
 Length of train_set is : 42000  and shape is:  (42000, 785)
 
 Length of test_set is : 28000  and shape is:  (28000, 784)


# 1. Primary checks and EDA.

In [4]:
# Confirm images represented as numeric dataframes from both train and test sets.

train_set.head(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test_set.head(10)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_set.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


**Plotting a few use-cases:**

In [7]:
# Examining % spread of digit labels.

pie1 = plex.pie(train_set, names = "label",title = "<b> Digit Labels </b>",hole = 0.4, template = "ggplot2",width=1200, height=600)
pie1.show()

In [8]:
# Examining counts on a bar plot.

value_counts = train_set['label'].value_counts()

bar3 = plex.bar(x=value_counts.index,y=value_counts.values,template='gridon', title="Value counts of digits",pattern_shape_sequence=["."])
bar3.show()

# 2. Preprocessing Data

In [9]:
# Splitting X and y from the train_set.

y_train = train_set['label']
X_train = train_set.drop(columns=['label'])
X_test = test_set.copy()

print(" \n X_train shape is: ",X_train.shape)
print(" \n X_test shape is: ",X_test.shape)
print(" \n y_train shape is: ",y_train.shape)

# Shapes are good. Onto normalizing the data. 
# Normalizing the color values from minimum 0 ( Black) to maximum 255 ( White)

X_train = X_train / 255 
X_test = X_test / 255

 
 X_train shape is:  (42000, 784)
 
 X_test shape is:  (28000, 784)
 
 y_train shape is:  (42000,)


# 3. Building a NN

In [10]:
# Defining early stopping parameters for epochs.

early_stopper = EarlyStopping( patience=10,
                              min_delta=0.001,
                              restore_best_weights=True)

first_nn = keras.Sequential([
    keras.layers.Dense(28,input_shape=[784]),
    keras.layers.Dense(392, activation='relu'),
    keras.layers.Dense(28, activation='relu'),
    keras.layers.Dense(10,activation='sigmoid')
])

first_nn.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')

first_nn.fit(X_train,y_train,
             epochs=30,               
             callbacks=[early_stopper]) 

2021-09-28 17:00:24.282459: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-28 17:00:24.285300: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-28 17:00:24.285329: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-28 17:00:24.285353: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (28ce0502e0a9): /proc/driver/nvidia/version does not exist
2021-09-28 17:00:24.286246: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Epoch 1/30
1313/1313 [==============================] - 3s 2ms/step - loss: 0.5837 - accuracy: 0.8204
Epoch 2/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.1610 - accuracy: 0.9486
Epoch 3/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.1240 - accuracy: 0.9616
Epoch 4/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0987 - accuracy: 0.9679
Epoch 5/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0759 - accuracy: 0.9755
Epoch 6/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0633 - accuracy: 0.9791
Epoch 7/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0541 - accuracy: 0.9817
Epoch 8/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0448 - accuracy: 0.9850
Epoch 9/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0412 - accuracy: 0.9864
Epoch 10/30
1313/1313 [==============================] - 2s 2ms/step - loss: 0.038

# 3.1 Predict Manually

In [11]:
# Predicting manually -  Given a random number, select a number(index) from the X_train_input array.

X_train_array = ny.array(X_train)

X_train_input = X_train_array.reshape(X_train_array.shape[0], 28, 28)

plex.imshow(X_train_input[32])

In [12]:
y_pred = first_nn.predict(X_train)
y_pred[32]

array([5.62802325e-06, 1.17397285e-04, 3.60101461e-04, 1.86714530e-03,
       1.00000000e+00, 3.11881304e-04, 2.18003988e-04, 9.98903275e-01,
       6.89360499e-03, 9.73331928e-01], dtype=float32)

In [13]:
print("\n Number predicted is: ", ny.argmax(y_pred[32]) ) # Did the number match the image ?


 Number predicted is:  4


In [14]:
# Actual predictions on X_test

predictions = ny.argmax(first_nn.predict(X_test), axis=1)

# reading submission file.
digits_submission = ps.read_csv('../input/digit-recognizer/sample_submission.csv')
digits_submission.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [15]:
digits_submission['Label']  = predictions
digits_submission.to_csv('submission.csv',index=False)
digits_submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


# 4. Observations

1. Building a Neural network without scaling the color values was easy but loss % change was low. Below are scores:

Epoch 14/15
1313/1313 [==============================] - 2s 1ms/step - loss: 4.5935 - accuracy: 0.9004

Epoch 15/15
1313/1313 [==============================] - 2s 1ms/step - loss: 4.7254 - accuracy: 0.8979


2. After scaling the dataframe scores seemed better:

Epoch 14/15
1313/1313 [==============================] - 2s 1ms/step - loss: 0.2308 - accuracy: 0.9340

Epoch 15/15
1313/1313 [==============================] - 2s 1ms/step - loss: 0.2282 - accuracy: 0.9354


3. With just one layer with 784 inputs and 10 outputs(units) the above scores were generated. Added 2 dense layers with half of input_shape.

Epoch 29/30
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0151 - accuracy: 0.9947

Epoch 30/30
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0156 - accuracy: 0.9955


4. Suggestions and comments are welcome !


**Thank you for your time !**